<a href="https://colab.research.google.com/github/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter/blob/main/AGnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-94d85bdf-1dc2-e28c-f0a4-b525a3db0f07)


#install pacakges

In [2]:
%%capture
!python3 -m pip install -U pip
!python3 -m pip install -U setuptools wheel
!python3 -m pip install -U "mxnet_cu110<2.0.0"
!python3 -m pip install autogluon

In [3]:
%%capture
!pip install -U transformers
!pip install catboost
!pip install xgboost
!pip install lightgbm

# download data

In [4]:
import os
if os.path.isdir('/content/Profiling-Hate-Speech-Spreaders-on-Twitter') is False:
    !git clone https://github.com/talhaanwarch/Profiling-Hate-Speech-Spreaders-on-Twitter.git

In [5]:
%%capture 
if os.path.isdir('train') is False:
  !unzip /content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/train.zip
if os.path.isdir('test') is False:
  !unzip /content/Profiling-Hate-Speech-Spreaders-on-Twitter/data/test.zip

# prepare data

## train data

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df=pd.read_csv('train/en/truth.txt',sep=':::',header=None,engine='python')
df.columns=['id','label']

In [7]:
df.label.value_counts()

1    100
0    100
Name: label, dtype: int64

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocessing(text):
    text=text.str.replace('\d+', '')
    text=text.str.replace('RT','')
    text=text.str.replace('#USER#','')
    text=text.str.replace('_','')
    text=text.str.replace('#URL#','')
    text= text.str.lower()
    text = text.str.replace('[^\w\s]','')
    #text = text.apply(lambda x : [lemmatizer.lemmatize(y) for y in w_tokenizer.tokenize(x)])
    #text = text.apply(lambda x: [item for item in x if item not in stop])
    #text = text.apply(lambda x : " ".join(x))
    return text

In [10]:
df.id='train/en/'+df.id.astype(str)+'.xml'
df.head()

,id,label
0,train/en/639b8e5e6a527d494c85d8f5704b1a01.xml,0
1,train/en/f2b1fc84c500c38a93522efbd422b559.xml,0
2,train/en/10b2d013382e1fb3c9414ea28329f258.xml,0
3,train/en/26644d1348fc1122e8c5ef45d6bc84fa.xml,0
4,train/en/4d4c5dcbfe38d0d33a0d1b1419952ca8.xml,0


In [11]:
len(df)

200

In [12]:
import xml.etree.ElementTree as ET
def reader(df,ground=True):
  data=[]
  for x in df.iterrows():
      
      tree = ET.parse(x[1].id)
      root = tree.getroot()
      text=[x.text for x in root[0]]
      if ground:
        label=[x[1].label]*len(text)
        data.append(pd.DataFrame(zip(text,label),columns=['text','label']))
      else:
        data.append(pd.DataFrame(text,columns=['text']))
  return data

In [13]:
data=reader(df)

In [14]:
data[0].head()

,text,label
0,RT #USER#: Funny how “15 days to slow the spre...,0
1,RT #USER#: Why did Minneapolis just give Georg...,0
2,"RT #USER#: To be fair, he has done a lot of un...",0
3,RT #USER#: President Trump got us the #HASHTAG...,0
4,RT #USER#: Is the case against former officer ...,0


In [15]:

datax=data.copy()
for d in range(len(data)):
  datax[d].text=preprocessing(data[d].text)


In [16]:
datax[0].head()

,text,label
0,funny how days to slow the spread turned in...,0
1,why did minneapolis just give george floyds ...,0
2,to be fair he has done a lot of undercover w...,0
3,president trump got us the hashtag vaccine p...,0
4,is the case against former officer derek cha...,0


In [17]:
len(datax[0])

200

In [18]:
doc_text,doc_label,=[],[]
for i in range(len(datax)):
  doc_text.append(list(datax[i].text))
  doc_label.append(int(datax[i].mean()))

In [19]:
len(doc_text),len(doc_label)

(200, 200)

## test data

In [20]:
from glob import glob
test_path=pd.DataFrame(glob('test/en/*.xml'),columns=['id'])
test_path.head()

,id
0,test/en/f2973063f16c0698a6de86c55b5f9ad6.xml
1,test/en/be71e41921653e523aceb555727b61c7.xml
2,test/en/d0235c3e74f79298fc2b6d2427c2060b.xml
3,test/en/314d4bf068314367bc83dc5ae3757e13.xml
4,test/en/fdd27881da241cff7115c1e89caba6dd.xml


In [21]:
test_data=reader(test_path,ground=False)

In [22]:

test_datax=test_data.copy()
for d in range(len(test_datax)):
  test_datax[d].text=preprocessing(test_datax[d].text)


In [23]:
test_doc=[]
for i in range(len(test_datax)):
  test_doc.append(list(test_datax[i].text))

# generate embeddings

In [24]:
from transformers import AutoTokenizer, AutoModel
from torch.nn.utils.rnn import pad_sequence
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",)
model = AutoModel.from_pretrained("bert-base-uncased",output_hidden_states=True)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
device='cuda'
model=model.to(device)

In [26]:
#get data from last hidden layer
def create_embeddings(doc_text,max_length=20):
  doc_embeddings=[]
  for text in doc_text:
    inputs=tokenizer(text,max_length=max_length,padding=True,truncation=True)
    text_seq = torch.tensor(inputs['input_ids']).to(device)
    text_mask = torch.tensor(inputs['attention_mask']).to(device)
    model.eval()
    with torch.no_grad():
      outputs= model(text_seq,attention_mask=text_mask)
      hidden_states=outputs.hidden_states
      emb=torch.mean(hidden_states[11], 1)
      doc_embeddings.append(emb)
  x=torch.mean(torch.stack(doc_embeddings),1)
  x=np.array(x.cpu())
  return x

In [27]:
%%time
x=create_embeddings(doc_text)

CPU times: user 44.3 s, sys: 185 ms, total: 44.5 s
Wall time: 43.4 s


In [28]:
y=doc_label
len(x),len(y)

(200, 200)

In [29]:
%%time
test=create_embeddings(test_doc)
print(len(test))

100
CPU times: user 22.9 s, sys: 41.4 ms, total: 22.9 s
Wall time: 22.4 s


# AutoGluon

In [30]:
data=pd.concat([pd.DataFrame(x),pd.DataFrame(y)],axis=1)
data.columns=['col_{}'.format(i) for i in range(x.shape[1])]+['Label']
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_729,col_730,col_731,col_732,col_733,col_734,col_735,col_736,col_737,col_738,col_739,col_740,col_741,col_742,col_743,col_744,col_745,col_746,col_747,col_748,col_749,col_750,col_751,col_752,col_753,col_754,col_755,col_756,col_757,col_758,col_759,col_760,col_761,col_762,col_763,col_764,col_765,col_766,col_767,Label
0,-0.060916,-0.060502,0.222756,-0.038621,-0.010247,-0.277739,0.054364,0.456214,-0.124440,-0.271279,0.242191,-0.168386,0.024935,0.167152,-0.271055,0.509920,0.021081,0.274331,-0.005994,0.204662,0.220118,0.032259,-0.119568,-0.124427,0.206190,0.060762,-0.187751,-0.117460,-0.292196,0.269815,0.277651,0.131913,0.206684,-0.096074,-0.124346,-0.250814,-0.017125,0.092924,-0.267352,0.241767,...,-0.236371,-0.100535,0.272637,-0.126820,-0.400522,-0.106810,-0.060659,-0.023552,0.212457,-0.181799,0.234672,0.034553,-0.069146,-0.148325,-0.127386,-0.067677,0.104943,0.008009,0.072241,0.069811,0.180995,-0.177055,-0.259657,0.236546,0.199358,-0.102678,-0.029406,-0.123981,-0.186943,-0.138305,-0.215604,-0.051728,-0.048241,0.118074,0.068692,-0.230986,-0.393949,-0.087446,-0.067072,1
1,0.013039,0.061095,0.308357,-0.077782,-0.101299,-0.142656,0.131202,0.612364,-0.019832,-0.329374,0.178789,-0.080739,0.082751,0.190460,-0.247831,0.451960,0.117547,0.326252,0.103056,0.148600,0.279433,0.052370,-0.256816,-0.205160,0.238519,0.212437,-0.228373,-0.102566,-0.335891,0.233442,0.244766,0.066441,0.117970,0.034631,-0.137634,-0.044614,-0.101110,0.007064,-0.344338,0.198573,...,-0.131852,-0.217642,0.152334,-0.110806,-0.269789,-0.140282,0.054728,-0.100997,0.332741,-0.195018,0.357707,-0.019838,-0.331002,-0.145794,-0.045196,-0.081138,-0.196560,0.131324,-0.096196,0.080504,0.030123,-0.315172,-0.336746,0.254149,0.214817,-0.095211,-0.099911,-0.262146,-0.203977,-0.132895,-0.369453,-0.077970,-0.022180,-0.039697,0.080661,-0.080027,-0.290338,-0.010375,-0.073959,1
2,-0.021055,-0.173784,0.186329,0.008758,-0.023974,-0.212143,0.025528,0.546581,-0.044222,-0.340376,0.131200,-0.043602,0.066705,0.129648,-0.203539,0.359138,-0.038409,0.318968,0.145757,0.127798,0.179783,0.064976,-0.201818,-0.182283,0.216703,0.117334,-0.061191,-0.014881,-0.542122,0.285893,0.335574,0.134865,0.259937,0.060966,-0.149418,-0.190508,-0.060077,0.250581,-0.443562,0.248681,...,-0.190377,-0.034363,0.167854,-0.220698,-0.374756,-0.221255,0.112621,-0.131392,0.318893,-0.289303,0.335746,0.019665,-0.139731,-0.236715,0.007882,-0.052243,-0.114845,0.095247,-0.003892,-0.044400,0.094224,-0.093771,-0.462861,0.193112,0.139246,-0.061216,-0.015146,-0.240348,-0.131825,-0.072950,-0.350931,-0.063833,0.064202,0.114927,0.115974,-0.074216,-0.353272,-0.082687,-0.037013,0
3,0.023635,0.067840,0.325173,-0.080089,-0.102563,-0.171877,0.112222,0.601954,-0.008847,-0.401733,0.156545,-0.043371,0.106803,0.174637,-0.213731,0.414555,0.110742,0.332099,0.127925,0.124926,0.266109,0.069558,-0.239082,-0.220760,0.263046,0.216456,-0.266520,-0.134086,-0.362071,0.276581,0.202446,0.034499,0.096662,0.057434,-0.162921,-0.014571,-0.116408,0.102767,-0.366734,0.134396,...,-0.084320,-0.206455,0.189938,-0.170704,-0.305640,-0.161934,0.106010,-0.142414,0.381790,-0.166898,0.354499,-0.025841,-0.340519,-0.176971,-0.064602,-0.111463,-0.172654,0.137292,-0.097064,0.074694,0.019507,-0.353251,-0.374266,0.285203,0.165368,-0.127045,-0.028807,-0.279689,-0.080784,-0.095926,-0.376545,-0.094100,0.020120,-0.078225,0.094399,-0.048818,-0.290314,-0.065547,-0.100407,0
4,0.057248,0.087196,0.417204,-0.083670,-0.149546,-0.142293,0.191529,0.595633,0.025340,-0.422448,0.121237,0.008685,0.116693,0.207466,-0.189136,0.451992,-0.001289,0.397060,0.155220,0.104650,0.326540,0.156941,-0.192788,-0.303176,0.242974,0.177639,-0.180634,-0.108984,-0.382166,0.351372,0.259992,0.045559,0.243831,0.102871,-0.172907,-0.048009,-0.116021,0.2

In [31]:
test_df=pd.DataFrame(test)
test_df.columns=['col_{}'.format(i) for i in range(test.shape[1])]
test_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39,...,col_728,col_729,col_730,col_731,col_732,col_733,col_734,col_735,col_736,col_737,col_738,col_739,col_740,col_741,col_742,col_743,col_744,col_745,col_746,col_747,col_748,col_749,col_750,col_751,col_752,col_753,col_754,col_755,col_756,col_757,col_758,col_759,col_760,col_761,col_762,col_763,col_764,col_765,col_766,col_767
0,0.065292,0.130962,0.312095,-0.125555,-0.115116,-0.102527,0.165735,0.569346,-0.031119,-0.487651,0.085751,-0.012549,0.039101,0.099278,-0.170304,0.475694,0.066658,0.400523,0.045677,0.149161,0.184343,0.127750,-0.168279,-0.232763,0.249590,0.254479,-0.192977,-0.109249,-0.414556,0.308195,0.303734,0.187180,0.171876,0.100622,-0.149850,-0.098993,0.051709,0.070413,-0.400671,0.321612,...,-0.189821,-0.023817,-0.231525,0.297786,-0.130105,-0.257663,-0.117976,0.149420,-0.168254,0.336325,-0.076671,0.228541,0.087887,-0.129244,-0.151576,-0.035447,-0.027074,-0.188494,0.038795,-0.265029,-0.001481,0.204445,-0.251798,-0.397929,0.160369,0.019442,-0.077531,-0.000511,-0.200781,-0.041115,-0.093149,-0.299660,-0.000484,0.059484,0.132730,0.114507,-0.004514,-0.243183,0.044420,-0.030861
1,-0.037613,-0.337574,0.239518,-0.081030,0.095761,-0.381824,-0.060326,0.332558,-0.283820,-0.161885,0.117458,-0.262703,-0.022409,0.175890,-0.176969,0.363772,-0.184882,0.355136,0.099598,0.100618,0.226659,0.017171,0.060798,-0.113058,0.209849,0.102618,0.011074,0.096027,-0.551804,0.192322,0.386268,0.134489,0.327850,-0.023012,-0.126237,-0.415216,-0.039691,0.100286,-0.227851,0.323735,...,-0.216006,-0.357152,0.068685,0.236276,-0.149925,-0.471562,-0.129042,-0.011927,0.012227,0.230680,-0.359140,0.304863,0.158982,-0.019429,-0.387436,0.054520,-0.074466,0.169215,0.056030,0.010047,-0.130596,0.249523,0.145141,-0.335341,0.242386,0.182804,0.049773,0.077531,-0.305582,-0.320237,-0.040202,-0.217032,0.104721,0.023172,0.282040,-0.069652,-0.217057,-0.444929,-0.146946,-0.060303
2,0.025136,-0.162270,0.213175,0.013490,-0.022161,-0.206344,0.043049,0.492201,-0.041096,-0.330985,0.142161,-0.085271,0.004717,0.145412,-0.219067,0.406614,-0.082015,0.290565,0.097302,0.145368,0.239947,0.097491,-0.187523,-0.206176,0.237590,0.182420,-0.088802,-0.126564,-0.486133,0.316576,0.276360,0.073294,0.252463,0.026992,-0.135007,-0.167214,-0.057213,0.121668,-0.322971,0.304161,...,-0.227252,-0.147719,-0.111663,0.253484,-0.147618,-0.333930,-0.167454,0.134778,-0.071234,0.286495,-0.231370,0.263061,0.079854,-0.151586,-0.149964,-0.035868,-0.006263,-0.000938,0.121968,-0.125791,-0.118690,0.147152,-0.128224,-0.401523,0.230557,0.162847,0.004047,-0.033898,-0.160781,-0.093900,-0.076430,-0.245507,-0.043845,0.038933,0.152348,0.078619,-0.062208,-0.311056,-0.082320,-0.070695
3,-0.069353,-0.048899,0.336302,-0.071292,-0.122794,-0.173350,0.131951,0.541700,-0.001292,-0.436529,0.189955,0.013303,0.144818,0.125889,-0.160800,0.431341,0.063810,0.378152,0.200411,0.058662,0.271755,0.102967,-0.237240,-0.261319,0.185950,0.244408,-0.260725,-0.105909,-0.373149,0.300826,0.152389,0.057719,0.176621,0.074684,-0.170620,-0.054195,-0.141244,0.097745,-0.363806,0.160746,...,-0.175566,-0.064771,-0.204180,0.240579,-0.185782,-0.256631,-0.271601,0.074254,-0.132022,0.447782,-0.180513,0.337425,-0.032905,-0.318529,-0.167260,-0.094197,-0.081709,-0.214091,0.145896,-0.113642,0.067639,0.016195,-0.246599,-0.384358,0.242567,0.153495,-0.125669,0.020994,-0.266984,-0.087892,-0.116395,-0.344607,-0.048895,0.038910,-0.033492,0.111118,-0.040606,-0.272553,-0.083607,-0.151875
4,0.077054,0.063156,0.245847,0.005221,-0.180295,-0.213427,0.137450,0.639700,-0.053647,-0.305522,0.190658,-0.122334,0.001214,0.240134,-0.313901,0.439687,-0.084962,0.308797,0.032967,0.180585,0.281594,0.111880,-0.258469,-0.178574,0.255789,0.165124,-0.115555,-0.108441,-0.417722,0.326026,0.354149,0.085615,0.149269,0.011796,-0.104251,-

In [32]:
%%time
from autogluon.tabular import  TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,accuracy_score
skf = StratifiedKFold(n_splits=5,shuffle=True)

acc=[]
test_pred=[]
for train_index, test_index in skf.split(data.iloc[:,0:-1], data.iloc[:,-1]):
  train=data.iloc[train_index,:]
  test=data.iloc[test_index,:]
  clf=TabularPredictor(label='Label',verbosity=0).fit(train, presets='best_quality')
  res=clf.leaderboard(test,silent=True)
  res['score_test']=res['score_test'].round(3)*100
  pred = clf.predict(test,model=res.model[0])
  accuracy=accuracy_score(test.Label,pred)
  acc.append(accuracy)
  print('best model ',res.model[0],' accuracy is ',accuracy)
  test_pred.append(clf.predict(test_df,model=res.model[0]))
print('average accuracy is',np.mean(acc))

best model  LightGBM_BAG_L1  accuracy is  0.7
best model  RandomForestEntr_BAG_L1  accuracy is  0.675
best model  KNeighborsDist_BAG_L1  accuracy is  0.75
best model  LightGBMXT_BAG_L1  accuracy is  0.8
best model  CatBoost_BAG_L1  accuracy is  0.775
average accuracy is 0.74
CPU times: user 21min 43s, sys: 17.4 s, total: 22min
Wall time: 13min 4s


In [33]:
ids=test_path.id.apply(lambda x:x.split('/')[-1].split('.')[0])
pred=np.mean(test_pred,axis=0)
lang=['en']*len(test_path)
pred=np.where(pred>0.5,1,0)

In [34]:
final=pd.DataFrame(zip(ids,lang,pred),columns=['author-id','lang','type'])
final.head()

,author-id,lang,type
0,f2973063f16c0698a6de86c55b5f9ad6,en,1
1,be71e41921653e523aceb555727b61c7,en,0
2,d0235c3e74f79298fc2b6d2427c2060b,en,0
3,314d4bf068314367bc83dc5ae3757e13,en,1
4,fdd27881da241cff7115c1e89caba6dd,en,1


In [35]:
final.to_csv('first.csv',index=False)

In [36]:
if os.path.isdir('output'):
  !rm -rf output
  !mkdir output
  !rm talha.zip
else:
  !mkdir output

rm: cannot remove 'talha.zip': No such file or directory


In [37]:
import pandas as pd
df=final.copy()
import pandas as pd
for row in df.index:
    xml = []
    xml.append('<author id="{}"'.format(df.loc[row,'author-id']))
    xml.append('lang="{}"'.format(df.loc[row,'lang']))
    xml.append('type="{}"'.format(df.loc[row,'type']))
    xml.append('/>')
    x='\n'.join(xml)
    myfile = open("output/{}.xml".format(df.loc[row,'author-id']), "w")
    myfile.write(x)
    myfile.close()
    

In [38]:
%%capture
!zip -r talha.zip output